### BIM to GIS Conversion

#### This Notebook imports revit files from BIM models and convert them to GIS geodatabase in prepartion for monitoring
#### Please implement the following steps:
1. Define Projection to PRS92
2. Export Feature to file geodatabase
3. Merge all feature layers
4. Delete unwanted fields
5. Add Fields
6. Calculate Field for Type
7. Calculate Field for SubType
8. Delete Fields

In [82]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'

fDataset = r'Station_AG'
path = os.path.join(workspace, fDataset)


In [83]:
# 1. Define Projection to PRS92
## Add revit files to the Contents panel
roof = r'Roofs'
framing = r'StructuralFraming'
foundation = r'StructuralFoundation'
column = r'StructuralColumns'

in_features = [framing, foundation, column]

sr = arcpy.SpatialReference("PRS 1992 Philippines Zone III")

for fl in in_features:
    # Define Projection first
    arcpy.DefineProjection_management(fl, sr)    


In [84]:
# 2. Export to file geodatabase

arcpy.FeatureClassToGeodatabase_conversion(in_features, path)


<Result 'C:\\Users\\oc3512\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\Station_AG'>

In [85]:
# 3. Merge all feature layers
arcpy.env.workspace = path
output_feature = "Station_Structure_merged"
arcpy.Merge_management(in_features, output_feature,"","")

<Result 'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb\\Station_AG\\Station_Structure_merged'>

In [86]:
# 4. Delete Fields
## 4.1 Obtain fields to be kept
fieldList = [f.name for f in arcpy.ListFields(output_feature)]

# find OBJECTID, as this is not deletable
reg = re.compile(r'OBJECTID*|Shape|Shape_Length|Shape_Area|Shape.STArea()|Shape.STLength()|GlobalID')
objField = list(filter(reg.match, fieldList))
keepFields = objField + ['Category', 'Bldg_Name', 'BldgLevel', 'OmniClassDescription', 'BldgLevel_Desc']

# Drop fields
dropFields = [e for e in fieldList if e not in keepFields]

## 4.2 Delete fields from feature layer
arcpy.DeleteField_management(output_feature, dropFields)

<Result 'Station_Structure_merged'>

In [87]:
# 5. Add Fields
addFields = ['Station', 'Type', 'SubType', 'Status', 'StartDate', 'TargetDate']

for field in addFields:
    if field in ('StartDate', 'TargetDate'):
        arcpy.AddField_management(output_feature, field, "DATE", "", "", "", field, "NULLABLE", "")
    else:
        arcpy.AddField_management(output_feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [88]:
# 6. Calculate Field for 'Type'
# codeblok for "Type"
codeblock = """
def reclass(category):
    if category == 'Structural Foundations':
        return 1
    elif category == 'Structural Columns':
        return 2
    elif category == 'Structural Framing':
        return 3
    elif category == 'Roofs':
        return 4
    else:
        return None"""

# codeblock for "SubType"
codeblock2 = """
def reclass(category):
    if category == 'Structural Foundations':
        return 1
    elif category == 'Structural Framing':
        return 2
    elif category == 'Structural Columns':
        return 4
    elif category == 'Roofs':
        return 5
    else:
        return None"""

calculateFields = ['Type', 'SubType']
for field in calculateFields:
    expression = "reclass(!{}!)".format("Category")
    if field == 'Type':
        arcpy.CalculateField_management(output_feature, field, expression, "PYTHON3", codeblock)
    else:
        arcpy.CalculateField_management(output_feature, field, expression, "PYTHON3", codeblock2)
        

In [89]:
# Assign Domains
domains = arcpy.da.ListDomains(workspace)
listDomain = [d.name for d in domains]
reg = re.compile(r'Station Structures*|Station*')
listFinal = list(filter(reg.match, listDomain))

listFinal

arcpy.AssignDomainToField_management(output_feature, 'Type', listFinal[0])
arcpy.AssignDomainToField_management(output_feature, 'SubType', listFinal[1])
arcpy.AssignDomainToField_management(output_feature, 'Status', listFinal[2])
arcpy.AssignDomainToField_management(output_feature, 'Station', listFinal[3])

<Result 'Station_Structure_merged'>

In [90]:
# Apply Symbology
symbolLyrx = r'C:/Users/oc3512/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structures.lyrx'
arcpy.ApplySymbologyFromLayer_management(output_feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [91]:
# Delete Fields
dropF = ['Category', 'Bldg_Name', 'OmniClassDescription', 'BldgLevel_Desc']
arcpy.DeleteField_management(output_feature, dropF)

<Result 'Station_Structure_merged'>